In [2]:
!pip install mysql-connector-python

In [3]:
import pandas as pd
import matplotlib.pyplot as plyt

In [15]:
import mysql.connector

con = mysql.connector.connect(user='hbs-student', password='super-secure-password?',
                              host='hbs-ds-060120.c1doesqrid0e.us-east-1.rds.amazonaws.com',
                              database='movies')

In [10]:
imdb_title_crew_df = pd.read_sql_query("""
SELECT * 
FROM movies.imdb_title_crew;
""",con)

In [16]:
imdb_title_crew_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   tconst     146144 non-null  object
 1   directors  146144 non-null  object
 2   writers    110261 non-null  object
dtypes: object(3)
memory usage: 3.3+ MB


In [11]:
imdb_title = pd.read_sql_query("""
SELECT * 
FROM movies.imdb_title_basics;
""",con)

In [12]:
imdb_title.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [17]:
imdb_people = pd.read_sql_query("""
SELECT * 
FROM movies.imdb_name_basics;
""",con)

In [18]:
imdb_ratings = pd.read_sql_query("""
SELECT 
	tconst,
    averagerating
FROM movies.imdb_title_ratings
""",con)

## Execution

In [19]:
imdb_test = imdb_title_crew_df['directors'].str.split(pat=',',expand=True)

In [20]:
imdb_test.head()

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,nm0712540,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,nm0002411,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,nm0000080,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,nm0611531,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,nm0765384,nm0749914,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [21]:
imdb_test.iloc[ : ,1].isna().sum() #Out of 146144 movies, 130417 have only 1 director --> delete all movies with multiple directors for our analysis

130417

In [22]:
imdb_test['tconst'] = imdb_title_crew_df['tconst']

In [23]:
imdb_test = imdb_test.drop(imdb_test.iloc[:,3:26], axis = 1)

In [24]:
imdb_title_crew_df['dir1'] = imdb_test[0]
imdb_title_crew_df['dir2'] = imdb_test[1]
imdb_title_crew_df['dir3'] = imdb_test[2]

In [25]:
imdb_crew = imdb_title_crew_df

In [26]:
imdb_crew.head()

,tconst,directors,writers,dir1,dir2,dir3
0,tt0063540,nm0712540,"nm0023551,nm1194313,nm0347899,nm1391276",nm0712540,None,None
1,tt0066787,nm0002411,None,nm0002411,None,None
2,tt0069049,nm0000080,"nm0000080,nm0462648",nm0000080,None,None
3,tt0069204,nm0611531,nm0347899,nm0611531,None,None
4,tt0100275,"nm0765384,nm0749914","nm1360635,nm0749914",nm0765384,nm0749914,None


In [27]:
writers = imdb_title_crew_df['writers'].str.split(pat=',',expand=True)

In [28]:
writers.iloc[:,3].isna().sum() #Out of 146144 movies, 140714 have only max 3 writers 

140714

In [29]:
imdb_crew['wri1'] = writers[0]
imdb_crew['wri2'] = writers[1]
imdb_crew['wri3'] = writers[2]

In [30]:
imdb = imdb_title.set_index('tconst').join(imdb_crew.set_index('tconst'))

In [31]:
nconst_to_names = imdb_people.loc[:, ['nconst','primary_name']]

In [32]:
imdb = imdb.join(nconst_to_names.set_index('nconst'),
          on='dir1',
          rsuffix='_names')

In [33]:
imdb = imdb.rename(columns={'primary_name':'dir1_name'})

In [34]:
imdb = imdb.join(nconst_to_names.set_index('nconst'),
          on='dir2',
          rsuffix='_names')

In [35]:
imdb = imdb.rename(columns={'primary_name':'dir2_name'})

In [36]:
imdb = imdb.join(nconst_to_names.set_index('nconst'),
          on='dir3',
          rsuffix='_names')

In [37]:
imdb = imdb.rename(columns={'primary_name':'dir3_name'})

In [38]:
imdb = imdb.join(nconst_to_names.set_index('nconst'),
          on='wri1',
          rsuffix='_names')

In [39]:
imdb = imdb.rename(columns={'primary_name':'wri1_name'})

In [40]:
imdb = imdb.join(nconst_to_names.set_index('nconst'),
          on='wri2',
          rsuffix='_names')

In [41]:
imdb = imdb.rename(columns={'primary_name':'wri2_name'})

In [42]:
imdb = imdb.join(nconst_to_names.set_index('nconst'),
          on='wri3',
          rsuffix='_names')

In [43]:
imdb = imdb.rename(columns={'primary_name':'wri3_name'})

In [44]:
imdb.shape

(146144, 19)

In [51]:
imdb = imdb.reset_index()

In [56]:
imdb = imdb.join(imdb_ratings.set_index('tconst'),
          on='tconst',
          rsuffix='_ratings')

In [5]:
imdb_principles = pd.read_sql_query("""
SELECT * 
FROM movies.imdb_title_principals
LIMIT 1000;
""",con)

In [6]:
imdb_principles.columns

Index(['tconst', 'ordering', 'nconst', 'category', 'job', 'characters'], dtype='object')

In [7]:
imdb_principles.head(30)

,tconst,ordering,nconst,category,job,characters
0,tt0111414,1,nm0246005,actor,,"[""The Man""]"
1,tt0111414,2,nm0398271,director,,
2,tt0111414,3,nm3739909,producer,producer,
3,tt0323808,10,nm0059247,editor,,
4,tt0323808,1,nm3579312,actress,,"[""Beth Boothby""]"
5,tt0323808,2,nm2694680,actor,,"[""Steve Thomson""]"
6,tt0323808,3,nm0574615,actor,,"[""Sir Lachlan Morrison""]"
7,tt0323808,4,nm0502652,actress,,"[""Lady Delia Morrison""]"
8,tt0323808,5,nm0362736,director,,
9,tt0323808,6,nm0811056,producer,producer,


In [148]:
imdb_principles.pivot_table(index=imdb_principles['tconst'],columns='category')

ordering                                                     \
category     actor   actress archive_footage cinematographer composer   
tconst                                                                  
tt0111414      1.0       NaN             NaN             NaN      NaN   
tt0323808      2.5  2.500000             NaN             9.0      8.0   
tt0417610      2.0  3.000000             NaN             NaN     10.0   
tt0469152      3.5  1.500000             NaN             9.0      8.0   
tt0473032      1.5       NaN             NaN             NaN      NaN   
...            ...       ...             ...             ...      ...   
tt1294688      3.0  2.000000             NaN             9.0      8.0   
tt1298644      3.5  1.500000             NaN             NaN      NaN   
tt1300159      3.0  2.000000             NaN             NaN      NaN   
tt1308748      2.0  2.666667             NaN             7.0      6.0   
tt1308754      2.0  2.000000             NaN             NaN      NaN   

                                                                    
category  director editor producer production_designer self writer  
tconst                                                              
tt0111414      2.0    NaN      3.0                 NaN  NaN    NaN  
tt0323808      5.0   10.0      6.5                 NaN  NaN    NaN  
tt0417610      5.0    NaN      8.0                 NaN  NaN    6.0  
tt0469152      5.0   10.0      7.0                 NaN  NaN    6.0  
tt0473032      3.0    NaN      NaN                 NaN  NaN    NaN  
...            ...    ...      ...                 ...  ...    ...  
tt1294688      5.0   10.0      6.5                 NaN  NaN    NaN  
tt1298644      5.0    NaN     10.0                 NaN  NaN    7.5  
tt1300159      5.0    NaN     10.0                 NaN  NaN    7.5  
tt1308748      5.0    NaN      NaN                 NaN  NaN    NaN  
tt1308754      NaN    NaN      NaN                10.0  NaN    NaN  

[142 rows x 11 columns]